In [1]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata,rrankdata
from pymcdm import weights as mcdm_weights

In [2]:
from prettytable import from_csv
with open("hep_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+---------+-----------+--------+-------+----------+---------+---------+-------------+--------------+--------------+
|                      | ACURACY | PRECISION | RECALL |  AUC  | F1-SCORE | FP_RATE | FN_RATE | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+---------+-----------+--------+-------+----------+---------+---------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.86  |    0.89   |  0.86  |  0.86 |   0.87   |   0.12  |   0.25  |     0.88    | 0.137931034  | 4.764051875  |
|    DECISION TREE     |   0.79  |    0.87   |  0.79  | 0.775 |   0.82   |   0.2   |   0.25  |     0.8     | 0.206896552  | 7.146091599  |
| GAUSSIAN NAIVE BAYES |   0.83  |    0.88   |  0.83  |  0.95 |   0.85   |   0.16  |   0.25  |     0.84    | 0.172413793  | 5.955071737  |
|         SVM          |   0.86  |    0.74   |  0.86  |  0.63 |   0.8    |    0    |    1    |      1      | 0.137931034  | 4.763969158  |
|    NEURAL NETWORK    |   

In [3]:
x_matrix, alt_names = mcdm.load("hep_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weight = [0.10909090908971901,0.16363636363821488, 0.18181818182165294,0.12727272727352062,0.14545454545314052,0.0363636363625124,0.0727272727239339,0.0545454545446777,0.09090909091046283,0.018181818182165292]
weights=np.array(weight)
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [4]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


1.0 0.8966491473212682
4.0 0.6104598933575721
3.0 0.8091838405026297
5.0 0.55579384354844
7.0 0.19274455877659924
2.0 0.8656421689581668
6.0 0.27260232025049475


In [5]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

1.0 0.0
4.0 0.42683834694010825
3.0 0.11613361221562823
5.0 0.6155354812338629
6.0 0.9411764705844635
2.0 0.04280562676310936
7.0 0.9680061636317323


In [6]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

1.0 0.6636363636368514
5.0 -0.10303030303220939
3.0 0.2909090909088263
4.0 0.07272727272960067
7.0 -0.7272727272737631
2.0 0.4939393939408844
6.0 -0.6909090909101901


In [7]:
from prettytable import PrettyTable
print("#########################BREAST DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################BREAST DATASET#######################
+----------------------+--------+---------------------+-------+---------------------+-----------+----------------------+
|        Model         | TOPSIS |        Value        | VIKOR |        VALUES       | PROMETHEE |        VALUES        |
+----------------------+--------+---------------------+-------+---------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  1.0   |  0.8966491473212682 |  1.0  |         0.0         |    1.0    |  0.6636363636368514  |
|    DECISION TREE     |  4.0   |  0.6104598933575721 |  4.0  | 0.42683834694010825 |    5.0    | -0.10303030303220939 |
| GAUSSIAN NAIVE BAYES |  3.0   |  0.8091838405026297 |  3.0  | 0.11613361221562823 |    3.0    |  0.2909090909088263  |
|         SVM          |  5.0   |   0.55579384354844  |  5.0  |  0.6155354812338629 |    4.0    | 0.07272727272960067  |
|    NEURAL NETWORK    |  7.0   | 0.19274455877659924 |  6.0  |  0.9411764705844635 |    7

In [8]:
ans=mcdm.rank(x_matrix, alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1],n_method="Linear2", w_vector=weight, s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
| LOGISTIC REGRESSION  |  0.7502680110284058 |
|    RANDOM FOREST     |  0.7381650212544082 |
| GAUSSIAN NAIVE BAYES |  0.7065729100781957 |
|    DECISION TREE     |  0.5647778698511088 |
|         SVM          |  0.5429572878081572 |
|     LINEAR MODEL     | 0.30312628547936676 |
|    NEURAL NETWORK    |  0.2694805198074564 |
+----------------------+---------------------+


In [9]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight,n_method="Linear1",s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
|    RANDOM FOREST     | 0.8321166094677085 |
| GAUSSIAN NAIVE BAYES | 0.8297097108677532 |
|    DECISION TREE     | 0.8116267292129197 |
|     LINEAR MODEL     | 0.8091329677456742 |
| LOGISTIC REGRESSION  | 0.8056930820474717 |
|    NEURAL NETWORK    | 0.7853858777398258 |
|         SVM          |        0.0         |
+----------------------+--------------------+
